In [1]:
from tensorflow.keras.layers import Conv2D, Dense, Reshape, Flatten, Conv2DTranspose, LeakyReLU, Dropout, Input
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from random import randint
import numpy as np
import matplotlib.pyplot as plt

### Latent Space

Points from the latent space. We can achieve this by calling the randn() NumPy function for generating arrays of random numbers drawn from a standard Gaussian. 

**The array of random numbers can then be reshaped into samples, that is n rows with 100 elements per row.**

In [2]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [3]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate ✬real✬ class labels (1)
    y = np.ones((n_samples, 1))
    return X, y 

In [4]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create ✬fake✬ class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

In [5]:
# load and prepare mnist training images
def load_real_samples():
    # load mnist dataset
    (trainX, _), (_, _) = load_data()
    # expand to 3d, e.g. add channels dimension
    X = np.expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X

### Define discriminator model

- Inputs: Image with one channel and 28 × 28 pixels in size.
- Outputs: Binary classification, likelihood the sample is real (or fake

The model is trained to minimize the binary cross-entropy loss function, appropriate for binary classification. 

In [6]:
def define_discriminator():
    input = Input(shape=(28,28,1))
    x = Conv2D(64, (3,3), strides=(2,2), padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.4)(x)
    x = Conv2D(64, (3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(x)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model = Model(inputs= input, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])
    return model 

In [7]:
model_discriminator = define_discriminator()

In [8]:
model_discriminator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 64)          0     

In [9]:
#Ge the data
(trainX, _), (_,_) = load_data()

In [10]:
# The images are 2D arrays of pixels and convolutional neural networks expect 3D arrays of
# images as input, where each image has one or more channels. 
# specify the final dimension for the channels-last image format.
X = np.expand_dims(trainX, axis=-1)

In [11]:
#scale the images. 
X = X.astype('float32')
X = trainX/255.0

### Define Generator

fake but plausible images of handwritten digits. It does this by taking a point from the latent space as input and outputting a square

The latent space is an arbitrarily defined vector space of Gaussian-distributed values, e.g. 100 dimensions.

- Inputs: Point in latent space, e.g. a 100 element vector of Gaussian random numbers.
- Outputs: Two-dimensional square grayscale image of 28 × 28 pixels with pixel values in [0,1]

In [12]:
latent_dim = 100

In [13]:
def define_generator(latent_dim):
    input = Input(shape=(latent_dim))
    n_nodes = 128*7*7
    x = Dense(n_nodes)(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((7, 7, 128))(x)
    #Upsample to 14x14
    x = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    #Upsample to  28x28
    x = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    #final
    output = Conv2D(1, (7,7), activation='sigmoid', padding='same')(x)
    model = Model(inputs=input, outputs=output)
    return model

In [14]:
model_generator = define_generator(latent_dim = 100)

In [15]:
model_generator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_1 (Dense)             (None, 6272)              633472    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 6272)              0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                           

### Training GANs

The weights in the generator model are updated based on the performance of the discriminator
model. When the discriminator is good at detecting fake samples, the generator is updated more,
and when the discriminator model is relatively poor or confused when detecting fake samples,
the generator model is updated less.

In [16]:
#define the models

d_model = define_discriminator()
g_model = define_generator(latent_dim)

In [17]:
#define gan training

def define_gan(g_model, d_model):
    #make weights in the discriminator not trainable
    d_model.trainable = False
    #Connect them
    model = Sequential()
    #add generator
    model.add(g_model)
    #add discriminator
    model.add(d_model)
    #compile the model
    opt = Adam(learning_rate=0.0002, decay=0.5)
    model.compile(loss='binary_crossentropy', optimizer = opt)
    return model    

In [18]:
#run the model
gan_model = define_gan(g_model, d_model)

In [19]:
gan_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 28, 28, 1)         1164289   
                                                                 
 model_2 (Functional)        (None, 1)                 40705     
                                                                 
Total params: 1,204,994
Trainable params: 1,164,289
Non-trainable params: 40,705
_________________________________________________________________


Training the composite model involves **generating a batch worth of points in the latent space via the generate latent points() function in the previous section, and class = 1 labels
and calling the train on batch() function.**

In [20]:
def train_gan(gan_model, latent_dim, n_epochs=100, n_batch=256):
    #manually enumerate epochs
    for i in range(n_epochs):
        #prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        #create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        #update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)

In [21]:
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
    # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
        #save plot to file
        filename = 'generated_plot_e%03d.png' % (epoch+1)
        plt.savefig(filename)
        plt.close()

In [22]:
#evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)


In [29]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epoch = int(dataset.shape[0]/n_batch)
    half_batch = int(n_batch/2)
    #manually enumerate epochs
    for i in range(n_epochs):
        #enumerate batches  over the training set
        for j in range(bat_per_epoch):
            #get random 'real' examples:
            X_real, y_real = generate_real_samples(dataset, half_batch)
            #get random fake examples:
            X_fake, y_fake, = generate_fake_samples(g_model, latent_dim, half_batch)
            #create training set for the discriminator
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            #update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X,y)
            #prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            #create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            #update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            #summarize loss on this batch
            print('>%d, %d/%d. d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epoch, d_loss, g_loss ))
        if (i+1)%10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [30]:
d_model = define_discriminator()

In [31]:
g_model = define_generator(latent_dim)

In [32]:
gan_model = define_gan(g_model, d_model)

In [33]:
dataset = load_real_samples()

In [34]:
train(g_model, d_model, gan_model, dataset, latent_dim)

4/4 [==============================] - 1s 180ms/step
>1, 1/234. d=0.697, g=0.726
4/4 [==============================] - 1s 167ms/step
>1, 2/234. d=0.692, g=0.738
4/4 [==============================] - 1s 254ms/step
>1, 3/234. d=0.686, g=0.752
4/4 [==============================] - 1s 169ms/step
>1, 4/234. d=0.680, g=0.763
4/4 [==============================] - 1s 266ms/step
>1, 5/234. d=0.674, g=0.779
4/4 [==============================] - 1s 288ms/step
>1, 6/234. d=0.667, g=0.796
4/4 [==============================] - 1s 257ms/step
>1, 7/234. d=0.660, g=0.809
4/4 [==============================] - 1s 179ms/step
>1, 8/234. d=0.650, g=0.827
4/4 [==============================] - 1s 174ms/step
>1, 9/234. d=0.648, g=0.848
4/4 [==============================] - 1s 166ms/step
>1, 10/234. d=0.639, g=0.869
4/4 [==============================] - 1s 168ms/step
>1, 11/234. d=0.630, g=0.891
4/4 [==============================] - 1s 158ms/step
>1, 12/234. d=0.622, g=0.913
4/4 [====================